In [1]:
import sys
sys.path.append("../../..")

from fridom.NonHydrostatic.ModelSettings import ModelSettings
from fridom.NonHydrostatic.Grid import Grid
from fridom.NonHydrostatic.Model import Model
from fridom.NonHydrostatic.Plot import PlotContainer
from fridom.NonHydrostatic.ModelPlotter import ModelPlotter
from fridom.NonHydrostatic.Source import PolarizedWaveMaker

In [3]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

class MyPlotter(ModelPlotter):
    def create_figure():
        fig = plt.figure(figsize=(8, 5), tight_layout=True)
        return fig
        
    def update_figure(fig, z, p, time, **kwargs):
        gs = gridspec.GridSpec(1, 2, width_ratios=[2, 1]) 

        ax1 = fig.add_subplot(gs[0])
        ax2 = fig.add_subplot(gs[1])
        grid = z.grid

        X, Y, Z = tuple(grid.X)
        im = PlotContainer.front_on_axis(ax1, z.b, X, Y, Z,
                                    0, -1, 1, "RdBu_r", None)
        plt.colorbar(im, ax=ax1).set_label("Buoyancy Perturbation")
        ax1.set_title(f"Front View,   t = {time:.3f}")

        ax2.plot(grid.N2_array, grid.x[2])
        ax2.set_title("Stratification N²")
        ax2.set_xlim([0, 12])
        ax2.set_ylabel("z")
        ax2.set_xlabel("N²")
        return

mset = ModelSettings(L=[3, 0.2, 3], N=[500, 2, 500], dt=0.01)
mset.gpu = True
mset.enable_source = True
mset.enable_nonlinear = False
mset.periodic_bounds = [False, True, False]
mset.enable_vid_anim = True
mset.vid_plotter = MyPlotter
mset.vid_anim_interval = 25
mset.vid_anim_filename = "ray_varying_N.mp4"
mset.enable_varying_N = True
grid = Grid(mset)
cp = grid.cp
grid.N2_array = 1 + 5 * (1 + cp.tanh((grid.x[2]-0.7*mset.L[2]) * 5))

wm = PolarizedWaveMaker(mset, grid, mask_pos=(0.5,None,0.5),
                        kx=80, kz=-40, amplitude=0.5, mask_width=(0.1,None,0.1))

model = Model(mset, grid)
model.source = wm

model.run(runlen=100)
model.show_video()

  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [01:59<00:00, 83.98it/s]
